<a href="https://colab.research.google.com/github/cipB14/Questify/blob/mod1/blooms_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install torch tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
os.makedirs("/root/.kaggle", exist_ok=True)
!kaggle datasets download -d vijaydevane/blooms-taxonomy-dataset


Dataset URL: https://www.kaggle.com/datasets/vijaydevane/blooms-taxonomy-dataset
License(s): apache-2.0
  0% 0.00/240k [00:00<?, ?B/s]
100% 240k/240k [00:00<00:00, 117MB/s]


In [3]:
import zipfile

zip_path = "blooms-taxonomy-dataset.zip"
extract_path = "blooms_taxonomy_data"

# Extract ZIP file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)


In [4]:
import transformers
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from datasets import Dataset, DatasetDict


In [5]:
from transformers import BertTokenizer
from transformers import DataCollatorWithPadding
from transformers import Trainer, TrainingArguments, BertForSequenceClassification

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [7]:
df = pd.read_csv("blooms_taxonomy_data/blooms_taxonomy_dataset.csv")
df.head()  # Display the first 5 rows

,Questions,Category
0,About what proportion of the population of the...,BT1
1,Correctly label the brain lobes indicated on t...,BT1
2,Define compound interest.,BT1
3,Define four types of traceability,BT1
4,Define mercantilism.,BT1


In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Category'])  # Encode labels
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)
from IPython.display import display
display(df)

Label Mapping: {'BT1': 0, 'BT2': 1, 'BT3': 2, 'BT4': 3, 'BT5': 4, 'BT6': 5}


,Questions,Category,label
0,About what proportion of the population of the...,BT1,0
1,Correctly label the brain lobes indicated on t...,BT1,0
2,Define compound interest.,BT1,0
3,Define four types of traceability,BT1,0
4,Define mercantilism.,BT1,0
...,...,...,...
8762,Distinguish between different types of soil st...,BT4,3
8763,Invent a blockchain-based solution for transpa...,BT6,5
8764,Compare the advantages and disadvantages of us...,BT4,3
8765,"Describe the purpose of the ""volatile"" keyword...",BT1,0


In [9]:

# ✅ Split into Train & Validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["Questions"], df["label"], test_size=0.2, random_state=42
)

# ✅ Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(pd.DataFrame({"text": train_texts, "label": train_labels}))
val_dataset = Dataset.from_pandas(pd.DataFrame({"text": val_texts, "label": val_labels}))

# ✅ Create DatasetDict
raw_datasets = DatasetDict({"train": train_dataset, "validation": val_dataset})

# ✅ Load Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# ✅ Define Tokenization Function
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# ✅ Apply Tokenization to Dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# ✅ Format Dataset for PyTorch
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
tokenized_datasets = tokenized_datasets.remove_columns(["__index_level_0__"])


# ✅ Print Tokenized Dataset
print(tokenized_datasets)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/7013 [00:00<?, ? examples/s]

Map:   0%|          | 0/1754 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7013
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1754
    })
})


In [10]:
from huggingface_hub import notebook_login
notebook_login()

In [11]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [13]:

# Load model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_mapping))

# Define training arguments
training_args = TrainingArguments(
    output_dir= "/content/drive/MyDrive/bert-blooms-taxonomy-classifier",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    push_to_hub=True,
    report_to="none"
)

# Define Trainer with data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,  # ✅ Adding dynamic padding
)

# Train model
trainer.train()

# Push to Hugging Face Hub
trainer.push_to_hub("cip29/blooms_bert")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-13-125d8df46e7f>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,0.611778
2,0.810600,0.536841
3,0.421600,0.572079


No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/cip29/bert-blooms-taxonomy-classifier/commit/ae343e4f4710e3cb48847b7db0d977d878c4a2e8', commit_message='cip29/blooms_bert', commit_description='', oid='ae343e4f4710e3cb48847b7db0d977d878c4a2e8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cip29/bert-blooms-taxonomy-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='cip29/bert-blooms-taxonomy-classifier'), pr_revision=None, pr_num=None)